# SQL 4 Data Science: Predict Project
## Bhejane Online Trading Store

### Student Starter Notebook

© Explore Data Science Academy

### Honour Code

I {**JUSTICE**, **OYEMIKE**}, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the EDSA honour code (https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).

Non-compliance with the honour code constitutes a material breach of contract.

<a id='Context'></a>
### 1. Context

The Bhejane trading store is an online retailer specialising in Covid essential items. The store has recently been struggling with the management of its database-related inventory system. Luckily for them, you've been hired as a consultant to fix the problem. 

Its time to flex your Ninja SQL skills! 
 
**Your mission, should you choose to accept it:**

You will receive a denormalised database consisting of two tables. Your task is to transform the database such that is in third normal form (3NF). To ensure that a consistent normalisation process is followed by each student, we have attached an Entity Relationship Diagram (ERD) which is a guideline on what tables need to be produced. Once you have normalised the database, you will be required to complete an MCQ test found under the 'Predict' tab of Athena.  

Though the normalisation process, you will be guided into acknowledging client (Bhenjane Trading) use-cases for the database, and dealing with data anomalies in SQL. 

![Bhejane company logo](https://raw.githubusercontent.com/Explore-AI/Pictures/master/SQL4DS/SQL_Predict/Bhejane.png)

<div align="center" >
    Bhejane, or the Black Rhino. Image by <a href="https://commons.wikimedia.org/wiki/File:Black_Rhino_(Diceros_bicornis)_browsing_..._(46584052962).jpg">Wikimedia Commons</a>
</div>

<a id='Imports'></a>
### 2. Imports
Please use the below command to install sql_magic, this is the package that will assist you with SQL syntax hightlighting.
* pip install sql_magic

Remember to start each new cell with " %%read_sql "

In [556]:
pip install sql_magic

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: sql_magic in c:\users\justy\anaconda3\lib\site-packages (0.0.4)



In [557]:
# DO NOT EDIT THIS CELL
import sqlite3
import csv
from sqlalchemy import create_engine
%load_ext sql_magic

# Load SQLite database
engine  = create_engine("sqlite:///bhejane.db")
%config SQL.conn_name ='engine'

The sql_magic extension is already loaded. To reload it, use:
  %reload_ext sql_magic


<a id='Data_description'></a>
### 3. Data description

The original database consist of 2 tables. 
* Product Table
* Transaction Table

The `Product` table consists of the stock of all inventory that Bhejane has on hand currently, or has had on hand historically. Items which are in this table are able to be purchased, and a record of all sales (transactions) in 2020 is notorised in the `Transactions` table. To link the tables - the `barcode` can be used. Any item in the `Transactions` table, must therefore appear in the `Products` table. 

In [558]:
# DO NOT EDIT THIS CELL
import pandas as pd
data_description = pd.read_excel('Data Description.xlsx')
data_description

,Table Name,Column Name,Desciption
0,Products,Width,Width of the product once assembled
1,Products,Length,Length of the product once assembled
2,Products,Height,Height of the product once assembled
3,Products,Barcode,The unique product identifier
4,Products,Quantity,Number of goods in stock
5,Products,Brand,Product brand name relating to product company
6,Products,NavigationPath,Navigation path to specific product
7,Products,Colour,Name default colour for the product
8,Products,StockCountry,Country where the stock was bought from
9,Products,ProductDescription,Descriptive product name


<a id='Setting_up'></a>
### 4. Setting up the database

In [559]:
#DO NOT EDIT THIS CELL
conn = sqlite3.connect('bhejane.db')
cursor = conn.cursor()

#### Creating both Products and Transaction Tables

In [560]:
%%read_sql
--DO NOT EDIT THIS CELL
DROP TABLE IF EXISTS "Products";
DROP TABLE IF EXISTS "Transactions";

CREATE TABLE "Products" (
    "Width"   REAL,
    "Length"  REAL,
    "Height"  REAL,
    "Barcode" VARCHAR(150),
    "Quantity" REAL,
    "Brand" VARCHAR(150), 
    "NavigationPath" VARCHAR(150),
    "Colour" VARCHAR(150),
    "StockCountry" VARCHAR(150),
    "ProductDescription" VARCHAR(150),
    "PackType" VARCHAR(150), 
    "Volume_litre" REAL, 
    "Warranty" VARCHAR(150), 
    "Weight_kg" REAL,
    "ItemDescription" VARCHAR(150), 
    "Price" REAL
);


CREATE TABLE "Transactions" (
    "CartID" INTEGER,
    "Barcode" VARCHAR(150), 
    "Total" REAL,
    "UserName" VARCHAR(150), 
    "InvoiceDate" DATETIME
);

Query started at 05:48:30 PM W. Central Africa Standard Time; Query executed in 0.00 m

#### Load Data into Product and Transaction tables

Although we have not yet covered python in the sprint at EXPLORE - you have additional resources at your disposal, one of which happens to be a data engineering expert, and she has created a brief script for you which can be used to extract the data from the csv files, and load it into a sqlite database. 

This will be included in your submission, so that the end-to-end processing of the data is visible, and repeatable for any additional consultants who may be brought on board at a later stage. 


In [561]:
# DO NOT EDIT CELL
#Load data into Product table
with open('bhejane_covid_essentials_Products.csv','r') as fin: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['Width'],i['Length'],i['Height'], i['Barcode'], i['Quantity'], i['Brand'], i['NavigationPath'], i['Colour'], i['StockCountry'], i['ProductDescription'],i['PackType'],i['Volume_litre'],i['Warranty'],i['Weight_kg'],i["ItemDescription"],i['Price']) for i in dr]

cursor.executemany("INSERT INTO Products VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);", to_db)
conn.commit()

with open('bhejane_covid_essentials_Transactions.csv','r') as fin: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['CartID'],i['Barcode'], i['Total'], i['UserName'], i['InvoiceDate']) for i in dr]

cursor.executemany("INSERT INTO Transactions VALUES (?, ?, ?, ?, ?);", to_db)
conn.commit()

<a id='ERD'></a>
### 5. Denormalized Database Tables

<br>

<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/SQL4DS/SQL_Predict/Denormalized_Tables.png" alt="Denormalized Tables" border="0">

#### 5.1 Explore the Denormalized Tables
Familiarize yourself with the data given in the two tables by writing your own SQL queries to explore properties of the dataset.i.e Look for data inconsistencies, anormalies, redundancies etc to guide your normalization process. 

This section will not be graded - think of it as your own scratch pad.



In [562]:
%%read_sql
-- #Use this section to explore the database by writing your own queries
SELECT * FROM Products


Query started at 05:48:32 PM W. Central Africa Standard Time; Query executed in 0.00 m

,Width,Length,Height,Barcode,Quantity,Brand,NavigationPath,Colour,StockCountry,ProductDescription,PackType,Volume_litre,Warranty,Weight_kg,ItemDescription,Price
0,,,,300507946,493.0,Hikvision,Computers & Tablets / Smart Home & Connected L...,,,Hikvision 1080P 2MP Turbo HD IR Bullet Camera,,0.0,Limited (6 months),,1 x Hikvision 1080P Bullet camera,399.0
1,,,,300507946,493.0,Hikvision,Computers & Tablets / Smart Home & Connected L...,,,Hikvision 1080P 2MP Turbo HD IR Bullet Camera,,0.0,Limited (6 months),,Manual,399.0
2,,,,10325354918,467.0,ZEE,Fashion / Accessories / Scarves,Grey,,ZEE 3-in-1 Unisex Gaiter,,0.0,Limited (6 months),,,139.0
3,,,,27131187035,275.0,Estee Lauder,Beauty / Luxury Beauty / Makeup / Face / Found...,Fresco,South Africa,Estee Lauder Double Wear Stay In Place Makeup,,0.0,Non-Returnable,,,655.0
4,,,,27131187035,275.0,Estee Lauder,Beauty / Luxury Beauty / Shop By Brand / Estee...,Fresco,South Africa,Estee Lauder Double Wear Stay In Place Makeup,,0.0,Non-Returnable,,,655.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1157,,,,TAL00035388021,220.0,Angelcare,Baby & Toddler / Nappies & Changing / Changing...,Blue,South Africa,Angelcare - Nappy Bin Refill - 3 Pack,,0.0,Limited (6 months),,x 3 Nappy Bin Refills,269.0
1158,,,,TAL00035388021,220.0,Angelcare,Baby & Toddler / Nappies & Changing / Changing...,Blue,South Africa,Angelcare - Nappy Bin Refill - 3 Pack,,0.0,Limited (6 months),,x 3 Nappy Bin Refills,269.0
1159,"21,3","19,8","3,2",TAL00035388407,130.0,Pampers,Baby & Toddler / Nappies & Changing / Wipes,White,South Africa,Pampers Complete Clean Baby Wipes - 6 x 64 - 3...,,0.0,Non-Returnable (6 months),,384 Complete Clean Wipes,159.0
1160,,,,TAL00035394505,410.0,RCT,Computers & Tablets / Smart Home & Connected L...,,South Africa,RCT 650VA Line Interactive UPS,,"10,72678",Limited (12 months),,1 x UPS,675.0


In [563]:
%%read_sql
SELECT *
FROM Transactions

Query started at 05:48:32 PM W. Central Africa Standard Time; Query executed in 0.00 m

,CartID,Barcode,Total,UserName,InvoiceDate
0,102,300507946,1523.0,DIMPHO,2020-07-02 0:00:00
1,1,43859499182,149.0,Hendrik,2020-08-05 0:00:00
2,2,614143543746,99.0,Faristha,2020-07-29 0:00:00
3,179,617566827837,3464.0,Zanele,2020-04-04 0:00:00
4,136,619659097318,3301.0,Junaid,2020-08-04 0:00:00
...,...,...,...,...,...
273,190,MPTAL72849953,126.0,Shameer,2020-07-18 0:00:00
274,191,MPTAL72849955,53.0,Eathon,2020-05-14 0:00:00
275,192,MPTAL72849955,757.0,Janet,2020-06-01 0:00:00
276,113,MPTALP14631,2535.0,Melandi,2020-08-11 0:00:00


### 6. Normalize the given Database tables to the 1st Normal Form (1NF)

Given the below below target ERD create new tables such the the database conforms to the 1st Normal Form

![1st Normal Form](https://raw.githubusercontent.com/Explore-AI/Pictures/master/SQL4DS/SQL_Predict/1stNF.png)


It is suggested that you create all your tables before attempting to populate them with data, this will help reduce errors that might creep up due to logical dependencies.

#### 6.1 Create the tables required for the 1st Normal Form
Use the above ERD sketch to create the required tables. Please label your tables as they appear in the ERD sketch

In [564]:
%%read_sql
--#Create tables required for 1NF
DROP TABLE IF EXISTS "Products_1NF";
DROP TABLE IF EXISTS "Transactions_1NF";

CREATE TABLE "Products_1NF"(
    "Barcode" VARCHAR(150),
    "NavigationPath" VARCHAR(150),
    "ItemDescription" VARCHAR(150),
    "Colour" VARCHAR(150),
    "ProductDescription" VARCHAR(150) NOT NULL,
    "Brand" VARCHAR(150),
    "Price" REAL NOT NULL,
    "Quantity" INTEGER NOT NULL,
    "PackType" VARCHAR(150),
    "Warranty" VARCHAR(150),
    "StockCountry" VARCHAR(150),
    "Weight_kg" REAL,
    "Volume_litre" REAL,
    "Length" REAL,
    "Width" REAL,
    "Height" REAL,
    PRIMARY KEY("Barcode","NavigationPath","ItemDescription")
);

CREATE TABLE "Transactions_1NF" (
    "CartID" INTEGER NOT NULL,
    "Barcode" VARCHAR(150) NOT NULL,
    "UserName" VARCHAR(150) NOT NULL,
    "InvoiceDate" DATETIME NOT NULL,
    "Total" REAL NOT NULL,
    FOREIGN KEY ("Barcode") REFERENCES "Products_1NF" ("Barcode"),
    PRIMARY KEY("CartID", "Barcode", "UserName")
);

Query started at 05:48:33 PM W. Central Africa Standard Time; Query executed in 0.00 m

#### 6.2 Populate the tables you have create in the above section.

Populate the tables such that the database conforms to the 1st Normal Form

In [565]:
%%read_sql
--#Populate the 1NF tables
DELETE FROM "Products_1NF";
DELETE FROM "Transactions_1NF";

INSERT INTO "Products_1NF" ("Barcode","NavigationPath","ItemDescription","Colour","ProductDescription","Brand",
                            "Price","Quantity","PackType","Warranty","StockCountry","Weight_kg",
                            "Volume_litre","Length","Width","Height")
SELECT DISTINCT
     Barcode
    ,NavigationPath
    ,ItemDescription
    ,Colour
    ,ProductDescription
    ,Brand
    ,Price
    ,Quantity
    ,PackType
    ,Warranty
    ,StockCountry
    ,Weight_kg
    ,Volume_litre
    ,Length
    ,Width
    ,Height 
FROM 
    Products;

INSERT INTO "Transactions_1NF"("CartID","Barcode","UserName","InvoiceDate","Total")
SELECT DISTINCT 
    CartID
    ,Barcode
    ,UserName
    ,InvoiceDate
    ,Total
FROM
    Transactions;
    

Query started at 05:48:35 PM W. Central Africa Standard Time; Query executed in 0.00 m

In [566]:
%%read_sql

SELECT *
FROM Transactions_1NF

Query started at 05:48:35 PM W. Central Africa Standard Time; Query executed in 0.00 m

,CartID,Barcode,UserName,InvoiceDate,Total
0,102,300507946,DIMPHO,2020-07-02 0:00:00,1523.0
1,1,43859499182,Hendrik,2020-08-05 0:00:00,149.0
2,2,614143543746,Faristha,2020-07-29 0:00:00,99.0
3,179,617566827837,Zanele,2020-04-04 0:00:00,3464.0
4,136,619659097318,Junaid,2020-08-04 0:00:00,3301.0
...,...,...,...,...,...
273,190,MPTAL72849953,Shameer,2020-07-18 0:00:00,126.0
274,191,MPTAL72849955,Eathon,2020-05-14 0:00:00,53.0
275,192,MPTAL72849955,Janet,2020-06-01 0:00:00,757.0
276,113,MPTALP14631,Melandi,2020-08-11 0:00:00,2535.0


In [567]:
%%read_sql

SELECT *
FROM Products_1NF


Query started at 05:48:35 PM W. Central Africa Standard Time; Query executed in 0.00 m

,Barcode,NavigationPath,ItemDescription,Colour,ProductDescription,Brand,Price,Quantity,PackType,Warranty,StockCountry,Weight_kg,Volume_litre,Length,Width,Height
0,300507946,Computers & Tablets / Smart Home & Connected L...,1 x Hikvision 1080P Bullet camera,,Hikvision 1080P 2MP Turbo HD IR Bullet Camera,Hikvision,399.0,493,,Limited (6 months),,,0.0,,,
1,300507946,Computers & Tablets / Smart Home & Connected L...,Manual,,Hikvision 1080P 2MP Turbo HD IR Bullet Camera,Hikvision,399.0,493,,Limited (6 months),,,0.0,,,
2,10325354918,Fashion / Accessories / Scarves,,Grey,ZEE 3-in-1 Unisex Gaiter,ZEE,139.0,467,,Limited (6 months),,,0.0,,,
3,27131187035,Beauty / Luxury Beauty / Makeup / Face / Found...,,Fresco,Estee Lauder Double Wear Stay In Place Makeup,Estee Lauder,655.0,275,,Non-Returnable,South Africa,,0.0,,,
4,27131187035,Beauty / Luxury Beauty / Shop By Brand / Estee...,,Fresco,Estee Lauder Double Wear Stay In Place Makeup,Estee Lauder,655.0,275,,Non-Returnable,South Africa,,0.0,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1157,TAL00035388021,Baby & Toddler / Nappies & Changing / Changing...,x 3 Nappy Bin Refills,Blue,Angelcare - Nappy Bin Refill - 3 Pack,Angelcare,269.0,220,,Limited (6 months),South Africa,,0.0,,,
1158,TAL00035388021,Baby & Toddler / Nappies & Changing / Changing...,x 3 Nappy Bin Refills,Blue,Angelcare - Nappy Bin Refill - 3 Pack,Angelcare,269.0,220,,Limited (6 months),South Africa,,0.0,,,
1159,TAL00035388407,Baby & Toddler / Nappies & Changing / Wipes,384 Complete Clean Wipes,White,Pampers Complete Clean Baby Wipes - 6 x 64 - 3...,Pampers,159.0,130,,Non-Returnable (6 months),South Africa,,0.0,"19,8","21,3","3,2"
1160,TAL00035394505,Computers & Tablets / Smart Home & Connected L...,1 x UPS,,RCT 650VA Line Interactive UPS,RCT,675.0,410,,Limited (12 months),South Africa,,"10,72678",,,


###  7. Convert the database into its 2nd Normal Form (2NF).

Please label your tables as they appear in the ERD sketch.

#### 7.1 Entity Relationship Diagram

![2nd Normal Form](https://raw.githubusercontent.com/Explore-AI/Pictures/master/SQL4DS/SQL_Predict/2ndNF.png)

#### 7.2 2NF Requirements
To transition from 1NF to 2NF, we need all columns in all tables to not have a partial dependancy on the PK of the table. This means that any tables which have a composite key e.g. `PRIMARY KEY("Barcode","NavigationPath","ItemDescription")` cannot have columns in the table which are dependant on only `Barcode`, `NavigationPath`, or `ItemDescription`. 

In moving to 2NF we will seperate out each of these as a primary key (for their own respective tables), and any columns dependant on one of these columns but not the others, will go into that table. The only columns remaining in the tables will be fully functionally dependant on the primary key of the table. For example, 

Thereby, we remove the partial dependancy, and will be in 2NF. We are instructed to create a unique identifier for the `Transactions_2NF`. 

**Second normal form notes**

<br>$\bullet$ Notice that `Total` is only dependant on `CartID` and not on `<CartID,barcode>`. This is a partial dependancy
<br>$\bullet$ Second normal form required no *partial functional dependancy* on the key. 
<br>$\bullet$ This can be achieved in 2 ways: create a unique identifier per row (so that there is a simple primary key), or break the table up into only the columns which are fully dependant on the composite key. 
<br>$\bullet$ We are going to opt for the easier solution, and create a unique simple primary key on the `Transactions_1NF`. We can create a unique primary key by concatenating `CartID||barcode`
<br>$\bullet$ NB - this approach does make it more challenging to get into 3NF! Which would be a good argument to opt for the latter approach. 

#### 7.3 Data Anomalies

You may stumble into an error when you insert the data which violates this constraint. You query this with the data owner, and you are told that there are three transactions in the database which were errors. You are informed that *all transactions in the database correspond to products are in the database*.

These entries were captured incorrectly, and must be removed. How will you deal with this? 

In [568]:
%%read_sql
select count(distinct barcode) from Transactions_1NF where barcode not in (select barcode from products_1NF)

Query started at 05:48:35 PM W. Central Africa Standard Time; Query executed in 0.00 m

,count(distinct barcode)
0,2


### <font color='turquoise'>Action: remove the affected entries with a `delete` query </font>

In [569]:
%%read_sql
delete from Transactions_1NF where barcode not in (select barcode from products_1NF)

Query started at 05:48:36 PM W. Central Africa Standard Time; Query executed in 0.00 m

In [570]:
%%read_sql
-- Should show no entries!
select count(distinct barcode) from Transactions_1NF where barcode not in (select barcode from products_1NF)

Query started at 05:48:36 PM W. Central Africa Standard Time; Query executed in 0.00 m

,count(distinct barcode)
0,0


#### 7.4 FK Requirements

_Note: sqlite does not always enforce foreign key constraints (and it is even more inconsistent with magic commands to enforce FK constraints!), so it is critical that you understand the purpose of the foreign key constraint and therefore are able to enforce the FK constraint yourself!_

To enforce data integrity, and to ensure that strange things cannot happen - we will have a FK constraint. A foreign key forces all values of the FK, to be limited to only the values which exist as entries in the PK column of the table for which the aforementioned table is a foreign key to. For example, we cannot sell a product (in our Transactions table) which we do not have in our Products table. 

You will need to work around this by removing the erroneous data from the tables before continuing, since this would violate the FK constraint between the Transactions_2NF and Products_2NF table otherwise. 

_Hint: Identify the observations in the Transactions_1NF table, and remove them from there._

#### 7.5 Client Use-Case Requirements

The data owner has advised that there were several instances of incorrect data capturing. Whenever data was partially captured, it was redone, but the partially-complete entries have not been removed from the database. 

For all tables which are going to be used, consider the table utility relative to the Products (Products_2NF) table. You are tasked with removing 'useless' entries as you take the data from 1NF to 2NF. For the sake of ease-of-use for the client, we will removing all rows in the 2NF tables `PackageContents_2NF` and `Colours_2NF` which have an empty value for the attributes that the client will use them for - these are the `ItemDescription` and `Colour` columns, respectively. 

_Hint: Make inserts into the other 2NF (`PackageContents_2NF` and `Colours_2NF`) tables using a `where` clause to remove instances of the value being an empty string, or a `null` value. This will need to be considered for the `ItemDescription` and for the `Colour` when data is inserted into the corresponding tables above (i.e. only insert data into the `PackageContents_2NF` which has a value for the `ItemDescription`, and only insert data into the `Colours_2NF` table which has a value for `Colour`_

### <font color='turquoise'>Action: Investigate these anomalies for the tables mentioned </font>

In [571]:
%%read_sql 

SELECT DISTINCT 
    ItemDescription,
    PackType,
    Warranty 
FROM
    Products
WHERE ItemDescription = '';

Query started at 05:48:36 PM W. Central Africa Standard Time; Query executed in 0.00 m

,ItemDescription,PackType,Warranty
0,,,Limited (6 months)
1,,,Non-Returnable
2,,,Limited (12 months)
3,,,Limited (120 months)
4,,,Limited (180 months)
5,,,Limited (18 months)
6,,,Supplier (12 months)
7,,Single,Limited (6 months)
8,,,Limited (24 months)
9,,,


### Let's do it! Construct the database above in second normal form

In [572]:
%%read_sql
--#Create tables required for 2NF

DROP TABLE IF EXISTS "Products_2NF";
DROP TABLE IF EXISTS "Transactions_2NF";
DROP TABLE IF EXISTS "Navigation_2NF";
DROP TABLE IF EXISTS "PackageContents_2NF";
DROP TABLE IF EXISTS "Colours_2NF";

CREATE TABLE Products_2NF(
    "RegistryID" INTEGER NOT NULL,
    "Barcode" VARCHAR(150) NOT NULL,
    "ProductDescription" VARCHAR(150) NOT NULL,
    "Brand" VARCHAR(150),
    "Price" REAL NOT NULL,
    "Quantity" INTEGER NOT NULL,
    "StockCountry" VARCHAR(150),
    "Weight_kg" REAL,
    "Volume_litre" REAL,
    "Length" REAL,
    "Width" REAL,
    "Height" REAL,
    "PathID" INTEGER,
    "ItemID" INTEGER,
    "ColourID" INTEGER,
    PRIMARY KEY(RegistryID AUTOINCREMENT),
    CONSTRAINT fk_nav FOREIGN KEY('PathID') REFERENCES Navigation_2NF ('PathID'),
    CONSTRAINT fk_pack FOREIGN KEY('ItemID') REFERENCES PackageContents_2NF ('ItemID'),
    CONSTRAINT fk_col FOREIGN KEY('ColourID') REFERENCES Colours_2NF ('ColourID') 
);


CREATE TABLE "Transactions_2NF" (
    "CartID||Barcode||UserName" VARCHAR(150) NOT NULL,
    "CartID" INTEGER NOT NULL,
    "Barcode" VARCHAR(150) NOT NULL,
    "UserName" VARCHAR(150) NOT NULL,
    "InvoiceDate" DATETIME NOT NULL,
    "Total" REAL NOT NULL,
    PRIMARY KEY("CartID||Barcode||UserName")
    CONSTRAINT fk_trans FOREIGN KEY('Barcode') REFERENCES Products_2NF ('Barcode')
);


CREATE TABLE "Navigation_2NF"(
    "PathID" INTEGER NOT NULL,
    "NavigationPath" VARCHAR(150),
    PRIMARY KEY("PathID" AUTOINCREMENT)
);

CREATE TABLE "Colours_2NF"(
    "ColourID" INTEGER NOT NULL,
    "Colour" VARCHAR(150),
    PRIMARY KEY("ColourID" AUTOINCREMENT)
);

CREATE TABLE "PackageContents_2NF"(
    "ItemID" INTEGER NOT NULL,
    "ItemDescription" VARCHAR(150),
    "PackType" VARCHAR(150),
    "Warranty" VARCHAR(150),
    PRIMARY KEY("ItemID" AUTOINCREMENT)
);


Query started at 05:48:38 PM W. Central Africa Standard Time; Query executed in 0.00 m

### Let's do it! _Populate_ the database above in second normal form

Firstly, let's consider the entries which are `null` and `= ''`. And we will only insert the relevant entries. 

In [573]:
%%read_sql
-- #Populate the tables so that they conform to 2NF

DELETE FROM "Products_2NF";
DELETE FROM "Transactions_2NF";
DELETE FROM "Navigation_2NF";
DELETE FROM "PackageContents_2NF";
DELETE FROM "Colours_2NF";


INSERT INTO "Navigation_2NF"("NavigationPath")
SELECT DISTINCT 
    NavigationPath 
FROM
    Products_1NF;
    
INSERT INTO "PackageContents_2NF"("ItemDescription","PackType","Warranty")
SELECT DISTINCT 
    ItemDescription,
    PackType,
    Warranty 
FROM
    Products_1NF
WHERE ItemDescription <> ''
        ;
    
INSERT INTO "Colours_2NF"("Colour")
SELECT DISTINCT 
    Colour
FROM
    Products_1NF
WHERE Colour <> ''
        ;

Query started at 05:48:40 PM W. Central Africa Standard Time; Query executed in 0.00 m

In [574]:
%%read_sql
SELECT *
FROM PackageContents_2NF

Query started at 05:48:41 PM W. Central Africa Standard Time; Query executed in 0.00 m

,ItemID,ItemDescription,PackType,Warranty
0,1,1 x Hikvision 1080P Bullet camera,,Limited (6 months)
1,2,Manual,,Limited (6 months)
2,3,1 x 350ml Free Air Duster,,Limited (6 months)
3,4,Wahl Home Pro Basic Corded 8 Piece Haircutting...,,Limited (6 months)
4,5,Wahl Easy Cut 15 Piece Complete Hair Clipper Kit,,Limited (6 months)
...,...,...,...,...
595,596,1 x Rosemary,,Limited (6 months)
596,597,1 x Peppermint,,Limited (6 months)
597,598,1 x Oximiter and Manual,,Limited (6 months)
598,599,x 3 Nappy Bin Refills,,Limited (6 months)


A more complicated scenario below as we insert into `Products_2NF`. We will insert from the Products_1NF table. We can use LEFT JOIN, because the data in each of these tables that `Products_1NF` has a FK to, originally came from `Products_1NF`, so we are gauranteed to get matches back. Normally however, what you would get from the FK restriction is the requirement to use _parent-table_ `LEFT JOIN` _child-table_. 

In [575]:
%%read_sql 

INSERT INTO "Products_2NF" ("PathID","ItemID","ColourID","Barcode","ProductDescription","Brand","Price",
                            "Quantity","StockCountry","Weight_kg","Volume_litre",
                            "Length","Width","Height")
SELECT
    PathID,
    ItemID,
    ColourID,
    Products_1NF.Barcode,
    ProductDescription,
    Brand,
    Price,
    Quantity,
    StockCountry,
    Weight_kg, 
    Volume_litre, 
    Length,
    Width,
    Height 
FROM 
    Products_1NF
LEFT JOIN Navigation_2NF 
    ON Products_1NF.NavigationPath = Navigation_2NF.NavigationPath 
LEFT JOIN PackageContents_2NF 
    ON Products_1NF.ItemDescription = PackageContents_2NF.ItemDescription
LEFT JOIN Colours_2NF 
    ON Products_1NF.Colour = Colours_2NF.Colour

Query started at 05:48:41 PM W. Central Africa Standard Time; Query executed in 0.01 m

Last but not least, we insert into the `Transactions_2NF`table. Recall that we created a new PK in this table, and pay attention to how we are inserting values into the table. 

In [576]:
%%read_sql 

INSERT INTO "Transactions_2NF"("CartID||Barcode||UserName","CartID","Barcode","UserName","InvoiceDate","Total")
SELECT DISTINCT 
    CartID||Transactions_1NF.Barcode||UserName
    ,CartID
    ,Transactions_1NF.Barcode
    ,UserName
    ,InvoiceDate
    ,Total
FROM
    Transactions_1NF
LEFT JOIN Products_2NF ON Transactions_1NF.Barcode = Products_2NF.Barcode ;


Query started at 05:48:41 PM W. Central Africa Standard Time; Query executed in 0.00 m

In [577]:
%%read_sql
SELECT *
FROM Transactions_2NF

Query started at 05:48:41 PM W. Central Africa Standard Time; Query executed in 0.00 m

,CartID||Barcode||UserName,CartID,Barcode,UserName,InvoiceDate,Total
0,143859499182Hendrik,1,43859499182,Hendrik,2020-08-05 0:00:00,149.0
1,2614143543746Faristha,2,614143543746,Faristha,2020-07-29 0:00:00,99.0
2,3619659141059JOHAN,3,619659141059,JOHAN,2020-05-20 0:00:00,75.0
3,4638142992763Sandra,4,638142992763,Sandra,2020-03-25 0:00:00,44.0
4,5638142992763Sheila,5,638142992763,Sheila,2020-08-03 0:00:00,44.0
...,...,...,...,...,...,...
270,191MPTAL72849955Eathon,191,MPTAL72849955,Eathon,2020-05-14 0:00:00,53.0
271,1926002305001472Janet,192,6002305001472,Janet,2020-06-01 0:00:00,757.0
272,1929336232000831Janet,192,9336232000831,Janet,2020-06-01 0:00:00,757.0
273,192MPTAL57588104Janet,192,MPTAL57588104,Janet,2020-06-01 0:00:00,757.0


<a id='Target_ERD'></a>
### 8. Convert the table into its 3rd Normal Form (3NF)

Please label your tables as they appear in the ERD sketch. In the same fashion as done in 2NF, note which tables need to have data inserted `where column <> '' and column is not null`.

<br>

![3rd Normal Form](https://raw.githubusercontent.com/Explore-AI/Pictures/master/SQL4DS/SQL_Predict/3rdNF.png)

In [578]:
%%read_sql

--#Create tables required for 2NF
DROP TABLE IF EXISTS "Transactions_3NF";
DROP TABLE IF EXISTS "Carts_3NF";
DROP TABLE IF EXISTS "Products_3NF";
DROP TABLE IF EXISTS "Users_3NF";
DROP TABLE IF EXISTS "Navigation_3NF";
DROP TABLE IF EXISTS "PackageContents_3NF";
DROP TABLE IF EXISTS "Colours_3NF";
DROP TABLE IF EXISTS "Brands_3NF";
DROP TABLE IF EXISTS "Locations_3NF";

CREATE TABLE "Products_3NF" (
    "RegistryID" INTEGER NOT NULL,
    "Barcode" VARCHAR(150) NOT NULL,
    "ProductDescription" VARCHAR(150) NOT NULL,
    "Price" REAL NOT NULL,
    "Quantity" INTEGER NOT NULL,
    "Weight_kg" REAL,
    "Volume_litre" REAL,
    "Length" REAL,
    "Width" REAL,
    "Height" REAL,
    "PathID" INTEGER,
    "ItemID" INTEGER,
    "ColourID" INTEGER,
    "BrandID" INTEGER,
    "LocationID" INTEGER,
    PRIMARY KEY("RegistryID" AUTOINCREMENT),
    FOREIGN KEY("PathID") REFERENCES "Navigation_3NF" ("PathID"),
    FOREIGN KEY("ItemID") REFERENCES "PackageContents_3NF" ("ItemID"),
    FOREIGN KEY("ColourID") REFERENCES "Colours_3NF" ("ColourID") ,
    FOREIGN KEY ("BrandID") REFERENCES "Brands_3NF" ("BrandID"),
    FOREIGN KEY ("LocationID") REFERENCES "Locations_3NF" ("LocationID")
);


CREATE TABLE "Carts_3NF" (
    "CartID" INTEGER NOT NULL,
    "InvoiceDate" DATETIME NOT NULL,
    "Total" REAL NOT NULL,
    PRIMARY KEY ("CartID")
);


CREATE TABLE "Users_3NF"(
    "UserID" INTEGER NOT NULL,
    UserName VARCHAR(50) NOT NULL,
    PRIMARY KEY("UserID" AUTOINCREMENT)
);


Query started at 05:48:45 PM W. Central Africa Standard Time; Query executed in 0.00 m

### <font color='turquoise'>Action: proceed with the construction of the remaining tables mentioned </font>

In [579]:
%%read_sql

CREATE TABLE "Transactions_3NF"(
    "CartID||Barcode||UserName" VARCHAR(150) NOT NULL,
    "CartID" INTEGER NOT NULL,
    "UserID" INTEGER NOT NULL,
    "Barcode" VARCHAR(150) NOT NULL,
    PRIMARY KEY("CartID||Barcode||UserName"),
    CONSTRAINT fk_Prod FOREIGN KEY('Barcode') REFERENCES 'Products_3NF' ('Barcode')
);
    
CREATE TABLE "Navigation_3NF"(
    "PathID" INTEGER NOT NULL,
    "NavigationPath" VARCHAR(150),
    PRIMARY KEY("PathID" AUTOINCREMENT)
);
    

CREATE TABLE "PackageContents_3NF"(
    "ItemID" INTEGER NOT NULL,
    "ItemDescription" VARCHAR(150),
    "PackType" VARCHAR(150),
    "Warranty" VARCHAR(150),
    PRIMARY KEY("ItemID" AUTOINCREMENT)
);

CREATE TABLE "Colours_3NF"(
    "ColourID" INTEGER NOT NULL,
    "Colour" VARCHAR(150),
    PRIMARY KEY("ColourID" AUTOINCREMENT)
);

CREATE TABLE "Brands_3NF"(
    'BrandID'INTEGER NOT NULL,
    'Brand'VARCHAR (150),
    PRIMARY KEY("BrandID" AUTOINCREMENT)
);
      
CREATE TABLE "Locations_3NF"(
    "LocationID" INTEGER NOT NULL,
    "StockCountry" VARCHAR(150),
    PRIMARY KEY("LocationID" AUTOINCREMENT)
);



Query started at 05:48:46 PM W. Central Africa Standard Time; Query executed in 0.00 m

In [580]:
%%read_sql
SELECT *
FROM Transactions_2NF


Query started at 05:48:46 PM W. Central Africa Standard Time; Query executed in 0.00 m

,CartID||Barcode||UserName,CartID,Barcode,UserName,InvoiceDate,Total
0,143859499182Hendrik,1,43859499182,Hendrik,2020-08-05 0:00:00,149.0
1,2614143543746Faristha,2,614143543746,Faristha,2020-07-29 0:00:00,99.0
2,3619659141059JOHAN,3,619659141059,JOHAN,2020-05-20 0:00:00,75.0
3,4638142992763Sandra,4,638142992763,Sandra,2020-03-25 0:00:00,44.0
4,5638142992763Sheila,5,638142992763,Sheila,2020-08-03 0:00:00,44.0
...,...,...,...,...,...,...
270,191MPTAL72849955Eathon,191,MPTAL72849955,Eathon,2020-05-14 0:00:00,53.0
271,1926002305001472Janet,192,6002305001472,Janet,2020-06-01 0:00:00,757.0
272,1929336232000831Janet,192,9336232000831,Janet,2020-06-01 0:00:00,757.0
273,192MPTAL57588104Janet,192,MPTAL57588104,Janet,2020-06-01 0:00:00,757.0


In [581]:
%%read_sql

--# Populate the tables to that they conform to the 3rd Normal Form
DELETE FROM "Products_3NF";
DELETE FROM "Transactions_3NF";
DELETE FROM "Carts_3NF";
DELETE FROM "Users_3NF";
DELETE FROM "Navigation_3NF";
DELETE FROM "PackageContents_3NF";
DELETE FROM "Colours_3NF";
DELETE FROM "Brands_3NF";
DELETE FROM "Locations_3NF";

INSERT INTO "Users_3NF" ("UserName")
SELECT DISTINCT UserName FROM Transactions_2NF;


Query started at 05:48:49 PM W. Central Africa Standard Time; Query executed in 0.00 m

### <font color='turquoise'>Action: proceed with the insertion into `Carts_3NF`.   </font>

Take note that there are duplicate values coming from `Transactions_2NF` - retain the `distinct` combinations of `CartID`,`InvoiceDate`,`Total` only.

In [582]:
%%read_sql

INSERT INTO "Carts_3NF"("CartID","InvoiceDate","Total")
SELECT DISTINCT 
    CartID,
    InvoiceDate,
    Total
FROM Transactions_2NF;



Query started at 05:48:49 PM W. Central Africa Standard Time; Query executed in 0.01 m

### <font color='turquoise'>Action: proceed with the insertion into the tables below. </font>

Similarly to the table above, mind the occurances of duplicate entries. These can be avoided by using `SELECT distinct` to avoid inserting redundant rows. 


In [583]:
%%read_sql

INSERT INTO "Locations_3NF" ("StockCountry")
SELECT DISTINCT
    StockCountry
FROM 
    Products_2NF
WHERE StockCountry <>'';
    

INSERT INTO "Brands_3NF" ("Brand")
SELECT DISTINCT
    Brand
FROM 
    Products_2NF
WHERE Brand <>'';


INSERT INTO "Colours_3NF" ("Colour")
SELECT DISTINCT
    colour
FROM 
    Colours_2NF 

WHERE Colour <>'' ;

INSERT INTO "PackageContents_3NF" ("ItemDescription","PackType","Warranty")
SELECT DISTINCT
    ItemDescription,
    PackType,
    Warranty
FROM 
    PackageContents_2NF
WHERE ItemDescription <>'' ;


INSERT INTO "Navigation_3NF" ("NavigationPath")
SELECT DISTINCT
    NavigationPath
FROM 
    Navigation_2NF
WHERE NavigationPath <>'';




Query started at 05:48:50 PM W. Central Africa Standard Time; Query executed in 0.00 m

### <font color='turquoise'>Action: proceed with the insertion into `Products_3NF`. </font>
Take note of the joins required for additional data contained in other tables. 

In [584]:
%%read_sql

INSERT INTO "Products_3NF" ("PathID","ItemID","ColourID","BrandID","LocationID",
                            "Barcode","ProductDescription","Price",
                            "Quantity","Weight_kg","Volume_litre",
                            "Length","Width","Height")
SELECT
    PathID,
    ItemID,
    ColourID,
    BrandID,
    LocationID,
    Barcode,
    ProductDescription,
    Price,
    Quantity,
    Weight_kg,
    Volume_litre,
    Length,
    Width,
    Height
FROM 
    Products_2NF
LEFT JOIN Brands_3NF          USING(Brand)
LEFT JOIN Locations_3NF       USING(StockCountry)


Query started at 05:48:51 PM W. Central Africa Standard Time; Query executed in 0.00 m

### <font color='turquoise'>Action: proceed with the insertion into `Transactions_3NF`.   </font>

Follow previous proceedure, and checks. 

In [585]:
%%read_sql

INSERT INTO "Transactions_3NF"("CartID||Barcode||UserName","CartID","Barcode","UserID")
SELECT 
    CartID||Barcode||UserName,
    CartID,
    Barcode,
    UserID
FROM 
    Transactions_2NF
LEFT JOIN Users_3NF USING(UserName);
    

Query started at 05:48:51 PM W. Central Africa Standard Time; Query executed in 0.01 m

### <font color='turquoise'>Action: Investigate the DB.   </font>

Note - the tables counts below are what you should arrive at. If you don't, reconsider how you have done the inserts above. Having the tables correctly formatted will allow to answer the questions more easily, and without errors.

| Table Name | Count |
| --- | --- |
| Brands_3NF | 232 |
| Carts_3NF |  190 |
| Colours_3NF | 17 |
| Locations_3NF | 2 |
| Navigation_3NF | 396 |
| PackageContents_3NF | 600 |
| Products_3NF | 1214 |
| Transactions_3NF | 275 |
| Users_3NF | 158 |

In [586]:
%%read_sql 

-- How many entries are here in each of the tables in the database now? 
select 'Products_3NF' as table_name, count(*) from Products_3NF
UNION
select 'Transactions_3NF' as table_name, count(*) from Transactions_3NF
UNION
select 'Users_3NF' as table_name, count(*) from Users_3NF
UNION
select 'Navigation_3NF' as table_name, count(*) from Navigation_3NF
UNION
select 'PackageContents_3NF' as table_name, count(*) from PackageContents_3NF
UNION
select 'Colours_3NF' as table_name, count(*) from Colours_3NF
UNION
select 'Brands_3NF' as table_name, count(*) from Brands_3NF
UNION
select 'Locations_3NF' as table_name, count(*) from Locations_3NF
UNION
select 'Carts_3NF' as table_name, count(*) from Carts_3NF


Query started at 05:48:51 PM W. Central Africa Standard Time; Query executed in 0.00 m

,table_name,count(*)
0,Brands_3NF,232
1,Carts_3NF,190
2,Colours_3NF,17
3,Locations_3NF,2
4,Navigation_3NF,395
5,PackageContents_3NF,600
6,Products_3NF,1214
7,Transactions_3NF,275
8,Users_3NF,158


<a id='MCQ_questions'></a>
## 9. MCQ Questions

Having completed the normalisation of the database, you may use the following cells to help you answer the MCQs for the predict. 

Q1) How many unique products does the company have?

In [598]:
%%read_sql

SELECT count(DISTINCT barcode) 
FROM Products_3NF


Query started at 05:49:17 PM W. Central Africa Standard Time; Query executed in 0.00 m

,count(DISTINCT barcode)
0,608


Q2) How many users bought from Bhejane in April 2020?

In [599]:
%%read_sql
-- Write your query here:
SELECT count(DISTINCT UserName),SUBSTR(InvoiceDate,1,7) AS 'Year'
FROM Users_3NF 
    LEFT JOIN Transactions_3NF USING(UserID) 
    LEFT JOIN Carts_3NF USING(CartID)
WHERE Year ='2020-04'

Query started at 05:49:17 PM W. Central Africa Standard Time; Query executed in 0.00 m

,count(DISTINCT UserName),Year
0,31,2020-04


Q3) How many users bought 3 or more items that cost more than R1000?

In [600]:
%%read_sql
-- Write your query here:
WITH CTE AS(
    SELECT UserName,  Total, count(Barcode)
    FROM Users_3NF 
        LEFT JOIN Transactions_3NF  USING(UserID)
        LEFT JOIN Carts_3NF         USING(CartID)
    GROUP BY CartID
    HAVING count(Barcode) >=3 AND  Total > 1000 
)

SELECT count(DISTINCT UserName )
FROM CTE;
    
    

Query started at 05:49:18 PM W. Central Africa Standard Time; Query executed in 0.00 m

,count(DISTINCT UserName )
0,11


Q4) Which user made the largest purchase on a single transaction?

In [601]:
%%read_sql
WITH CTE AS (
SELECT UserName,CartID, Total
    FROM Users_3NF 
        LEFT JOIN Transactions_3NF  USING(UserID)
        LEFT JOIN Carts_3NF         USING(CartID)
    GROUP BY CartID
    ORDER BY Total DESC
)
SELECT UserName,Total
FROM CTE
LIMIT 2;


Query started at 05:49:18 PM W. Central Africa Standard Time; Query executed in 0.00 m

,UserName,Total
0,Brigette,10999.0
1,Mandla,10999.0


Q5) How many components does the product: "5m Colour Changing RGB LED Strip Light" (MPTAL57588104) come with?

In [602]:
%%read_sql
SELECT count(*)
FROM Products_3NF
WHERE Barcode ='MPTAL57588104' AND ProductDescription ="5m Colour Changing RGB LED Strip Light"


Query started at 05:49:19 PM W. Central Africa Standard Time; Query executed in 0.00 m

,count(*)
0,8


Q6) How many brands are available at Bhejane?

In [603]:
%%read_sql
SELECT count( DISTINCT Brand) AS "How many brands are available at Bhejane?"
FROM Brands_3NF


Query started at 05:49:19 PM W. Central Africa Standard Time; Query executed in 0.00 m

,How many brands are available at Bhejane?
0,232


7) What is the price of the "Verimark - Floorwiz 2in1 Mop"?

In [604]:
%%read_sql
SELECT DISTINCT Price AS 'What is the price of the "Verimark - Floorwiz 2in1 Mop"?'
FROM Products_3NF
WHERE ProductDescription ="Verimark - Floorwiz 2in1 Mop"


Query started at 05:49:20 PM W. Central Africa Standard Time; Query executed in 0.00 m

,"What is the price of the ""Verimark - Floorwiz 2in1 Mop""?"
0,179.0


Q8) Calculate the package volume of the "Russell Hobbs - Slow Cooker" using the given dimensions

In [605]:
%%read_sql
SELECT DISTINCT (Length * Width * Height) AS 'package volume of the "Russell Hobbs - Slow Cooker"'
FROM Products_3NF
WHERE ProductDescription = 'Russell Hobbs - Slow Cooker'


Query started at 05:49:20 PM W. Central Africa Standard Time; Query executed in 0.00 m

,"package volume of the ""Russell Hobbs - Slow Cooker"""
0,26928.0


Q9) Which user made the most transactions in the Year 2020?

In [606]:
%%read_sql
WITH CTE AS (
SELECT UserName, count(CartID) AS 'Transaction_count', sum(Total) AS 'cumulative_total'
    FROM Users_3NF 
        LEFT JOIN Transactions_3NF  USING(UserID)
        LEFT JOIN Carts_3NF         USING(CartID)
        WHERE InvoiceDate BETWEEN '2020-01-01'AND '2020-12-31'
    GROUP BY UserName
    ORDER BY Transaction_count DESC
)
SELECT UserName, Transaction_count, cumulative_total
FROM CTE
LIMIT 1;


Query started at 05:49:21 PM W. Central Africa Standard Time; Query executed in 0.00 m

,UserName,Transaction_count,cumulative_total
0,Cornelis,9,2349.0


Q10) What is the total number of users that shop at Bhejane?

In [607]:
%%read_sql
SELECT count(DISTINCT UserName) AS 'What is the total number of users that shop at Bhejane?'
FROM Users_3NF


Query started at 05:49:21 PM W. Central Africa Standard Time; Query executed in 0.00 m

,What is the total number of users that shop at Bhejane?
0,158


Q11) What is the record count for the Colours_3NF Table?

In [608]:
%%read_sql
SELECT count(DISTINCT Colour) AS 'What is the record count for the Colours_3NF Table?'
FROM Colours_3NF


Query started at 05:49:22 PM W. Central Africa Standard Time; Query executed in 0.00 m

,What is the record count for the Colours_3NF Table?
0,17


Q12) What would the total price be if I had the following items in my cart?
* MPTAL57588104
* 5000394203921
* 6932391917652

In [609]:
%%read_sql

SELECT   sum(Price)
FROM Products_3NF
WHERE Barcode ='MPTAL57588104' OR Barcode ='5000394203921' OR Barcode ='6932391917652'

  




Query started at 05:49:23 PM W. Central Africa Standard Time; Query executed in 0.00 m

,sum(Price)
0,3982.0


Q13) What is the barcode of the most sold product?

In [610]:
%%read_sql
SELECT Barcode, count(Barcode) AS 'Item_count'
FROM Transactions_3NF
GROUP BY Barcode
ORDER BY Item_count DESC
LIMIT 1;


Query started at 05:49:23 PM W. Central Africa Standard Time; Query executed in 0.00 m

,Barcode,Item_count
0,4015400541813,4


Q14) What are the products of Cornelis’ cart on the 2020-06-28 0:00:00?

In [611]:
%%read_sql
SELECT  UserName,Barcode, ProductDescription 
FROM Products_3NF
INNER JOIN Transactions_3NF USING(Barcode)
    LEFT JOIN Users_3NF        USING(UserID)
    LEFT JOIN Carts_3NF        USING(CartID)
WHERE InvoiceDate = '2020-06-28 0:00:00'
GROUP BY Barcode



Query started at 05:49:24 PM W. Central Africa Standard Time; Query executed in 0.00 m

,UserName,Barcode,ProductDescription
0,Cornelis,6001106124069,Dettol Body Wash - Shower Gel - Original - 600ml
1,Cornelis,6003753002035AP,Vitality Aloe Vera Gel 100ml
2,Cornelis,MPTAL00555756,Sparq Active Resistance Loop Band - Set of 5 w...
3,Cornelis,MPTAL72193414,Protective Face Shield


Q15) Which users bought locally produced, black-coloured products on odd-numbered months of the year, and what was the total cost (rounded to the nearest integer) of the carts containing these products?

In [612]:
%%read_sql
WITH CTE AS (
SELECT  DISTINCT UserName AS "Name", SUBSTR(InvoiceDate,6,2) AS 'Month',ProductDescription, colour, CartID, Total
FROM Products_3NF
INNER JOIN Locations_3NF       USING(LocationID)
INNER JOIN Colours_3NF         USING(ColourID)
INNER JOIN Transactions_3NF    USING(Barcode)
    LEFT JOIN Users_3NF        USING(UserID)
    LEFT JOIN Carts_3NF        USING(CartID)
WHERE colour = 'Black' 
    AND Month IN ('01', '03', '05', '07', '09', '11')
    AND StockCountry = 'South Africa' 
)

SELECT  name, ProductDescription,Month, CartID, Total,colour
FROM CTE;
   
    





Query started at 05:49:24 PM W. Central Africa Standard Time; Query executed in 0.00 m

,Name,ProductDescription,Month,CartID,Total,colour
0,Cindy,RCT 2000VA Line Interactive UPS,03,7,1999.0,Black
1,Hester,Duracell Plus Alkaline AAA Batteries - 20 Pack,07,18,189.0,Black
2,Conrad,Addis - Rough Tote - 30 Litre,07,32,93.0,Black
3,Carel,JBL T500BT Wireless On-Ear Headphones - Black,05,82,799.0,Black
4,Lacin,Ausdom AW635 1080P Streaming Web Camera - Black,05,85,798.0,Black
5,Jeandre,Sodastream - Classic Carbonating Bottle 1 Litr...,07,87,179.0,Black
6,DIMPHO,Compatible Canon 737 Black Laser Toner Cartridge,07,102,1523.0,Black
